In [ ]:
!nvidia-smi

In [8]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [7]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 38.7 MB/s 
     |████████████████████████████████| 636 kB 54.3 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 3.3 MB 42.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
##
#COPA_train = pd.read_csv(PATH + '/COPA/SKT_COPA_Train.tsv',sep='\t')
#COPA_dev = pd.read_csv(PATH + '/COPA/SKT_COPA_Dev.tsv',sep='\t')
#COPA_test = pd.read_csv(PATH + '/COPA/SKT_COPA_Test.tsv',sep='\t')
##
# Load the dataset into a pandas dataframe.
PATH = '/content/drive/MyDrive/gh/klue/DATA'
df = pd.read_csv(PATH + '/COPA/SKT_COPA_Train.tsv', sep='\t')

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 3,080



,ID,sentence,question,1,2,Answer
727,728,음료수를 냉장고에 넣었다.,결과,음료수가 뜨겁다.,음료수가 차갑다.,2
124,125,우편함에 우편이 잔뜩 쌓여 있다.,원인,나에게 편지를 보낸 사람이 없다.,최근에 우편함을 확인하지 않았다.,2
1325,1326,파란색 색연필로 구름을 칠했다.,결과,구름은 파란색이다.,구름은 분홍색이다.,1
2219,2220,매운 음식을 먹었다.,결과,입이 얼얼해졌다.,입이 차가워졌다.,1
516,517,그 소년은 머리를 모두 밀었다.,원인,그는 파마를 하려고 한다.,그는 곧 입대한다.,2
1101,1102,남자아이가 풍선을 꽉 쥐었다.,결과,풍선에 바람이 가득찼다.,풍선이 터졌다.,2
643,644,언니의 옷을 몰래 입었다.,결과,언니가 나에게 화를 냈다.,언니가 수학여행을 갔다.,1
1024,1025,배에 타 노를 젓는다.,결과,배가 달고 맛있다.,배가 앞으로 나아가며 물결이 생긴다.,2
1887,1888,친구는 힌두교다.,결과,친구는 돼지고기를 먹는다.,친구는 소고기를 먹는다.,1
2320,2321,고추기름이 완성되었다.,원인,파와 기름을 볶았다.,고추와 기름을 볶았다.,2


In [4]:
df.loc[df.Answer == 1].sample(5)[['sentence', 'question','1','2','Answer']]

,sentence,question,1,2,Answer
2992,통 속의 물이 증발했다.,결과,통에 들어있는 물의 양이 줄어들었다.,통에 들어있는 물의 양이 늘어났다.,1
1149,그녀는 다섯 경기에서 연속으로 졌다.,결과,그녀의 상대가 그녀에게 미안해 한다.,그녀는 약물 도핑으로 고소당했다.,1
2837,서퍼들이 서핑을 중단했다.,결과,"서핑 도중, 상어를 보았다.",서핑하기 좋은 파도를 보았다.,1
403,검은 옷에 락스가 튀었다.,결과,옷에 물이 빠졌다.,옷의 색이 선명해졌다.,1
2818,"자고 일어났더니, 남자의 허리가 배겼다.",결과,남자가 침대 매트리스를 새로 샀다.,남자가 침대 시트를 새로 샀다.,1


In [5]:
# Get the lists of sentences and their labels.
sentences = df.sentence.values
question = df.question.values
choice1 = df['1'].values
choice2 = df['2'].values
labels = df.Answer.values

In [9]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/428 [00:00<?, ?B/s]

In [14]:
# Print the original sentence.
print(' Original: ', sentences[0] + " " + question[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0] + " " + question[0]))

# Print the sentence mapped to token ids.
#print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  이퀄라이저로 저음 음역대 소리 크기를 키웠다. 결과
Tokenized:  ['이', '##퀄', '##라이', '##저', '##로', '저', '##음', '음', '##역', '##대', '소리', '크기', '##를', '키웠', '##다', '.', '결과']


In [19]:
max_len = 0

# # For every sentence...
# for sent in sentences:

#     # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
#     input_ids = tokenizer.encode(sent, add_special_tokens=True)

#     # Update the maximum sentence length.
#     max_len = max(max_len, len(input_ids))

# print('Max sentence length: ', max_len)

for i in range(len(sentences)):
    input_ids = tokenizer.encode([sentences[i] + " " + question[i], sentences[i] + " " + question[i]], 
                                         [choice1[i], choice2[i]], add_special_tokens = True)
    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  3


In [17]:
input_ids = []
attention_masks = []

for i in range(len(sentences)):
    encoded_dict = tokenizer([sentences[i] + " " + question[i], sentences[i] + " " + question[i]], 
                                         [choice1[i], choice2[i]],
                                         add_special_tokens = True,
                                         max_length = 64,
                                         pad_to_max_length = True,
                                         return_attention_mask = True,
                                         return_tensors='pt',
                                         )
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])


# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  이퀄라이저로 저음 음역대 소리 크기를 키웠다.
Token IDs: tensor([    2,  1504,  3395,  4893,  2190,  2200,  1535,  2053,  1500,  2331,
         2104,  3856,  5585,  2138, 12871,  2062,    18,  3731,     3,  9763,
         3856,  2116,  1521,  7428,  2318,   859,  2359,  2062,    18,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [18]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

AssertionError: ignored